# Comparação de distâncias de segurança
Cálculo da flecha e distâncias verticais, para uma lista de obstáculos.

Determina-se a altura do suporte de acordo com a pior condição de obstáculo e risco térmico. Todas as alturas ficam referenciadas ao solo, supondo o mesmo vão para todos os obstáculos, somando quando for o caso a altura determinada.

Notebook dinâmico

In [1]:
# Rodar caso tenha acabado de baixar o repositório, ou após um update
#
# Utilizar ambiente virtual:
#    python -m venv .venv
#    . .venv/bin/activate
%pip install -r requirements.txt

In [2]:
from normaslt import types, nbr5422, nbr1985, iec, en, geral
from tabulate import tabulate
import pandas as pd
import numpy as np
from math import sqrt
from IPython.display import HTML, display
from ipywidgets import interactive, Dropdown, FloatSlider, FloatText

In [3]:
def tabelaDist(vn: types.tensao, cabo: geral.dbCabo, vao, t1, t5, t15):

    eds = 0.2
    t0 = 25.
    f15, T15 = geral.equacaoEstado(cabo.p, vao, cabo.T * eds, cabo.S, cabo.E, cabo.alfa1, t0, t15)
    f5, T5 = geral.equacaoEstado(cabo.p, vao, cabo.T * eds, cabo.S, cabo.E, cabo.alfa1, t0, t5)
    f1, T1 = geral.equacaoEstado(cabo.p, vao, cabo.T * eds, cabo.S, cabo.E, cabo.alfa1, t0, t1)
    obs = [types.obs.PEDESTRE,
      types.obs.MAQ_AGRICOLA,
      types.obs.RODOVIA,
      types.obs.FERROVIA_NAO_ELETRIFICADA,
      types.obs.FERROVIA_ELETRIFICADA,
      types.obs.SUPORTE_FERROVIA,
      types.obs.AGUAS_NAVEGAVEIS,
      types.obs.AGUAS_NAO_NAVEGAVEIS,
      types.obs.LINHA_TRANSMISSAO,
      types.obs.LINHA_TELECOM,
      types.obs.VEGETACAO_PERM,
      types.obs.CULT_AGRIC_PERM,
      types.obs.INSTALACAO_TRANSP]
    hobs = [3.9, 4., 5.4, 6.4, 9.7, 5, 20, 3.6, 20, 10, 10, 4, 10]
    Fsfl = 2.2
    kafl = 1
    dv1 = []
    dv2 = []
    dv3 = []
    dv4 = []
    
    for o, h in zip(obs, hobs):        
        dv1.append(nbr5422.distSegurancaVert(o, types.amp.TIPICA_NOMINAL, h, vn.nom, Fsfl, kafl))
        dv2.append(nbr5422.distSegurancaVert(o, types.amp.TIPICA_SOBRECORRENTE, h, vn.nom, Fsfl, kafl))
        dv3.append(nbr5422.distSegurancaVert(o, types.amp.LIMITE_NOMINAL, h, vn.nom, Fsfl, kafl))
        dv4.append(nbr5422.distSegurancaVert(o, types.amp.LIMITE_SOBRECORRENTE, h, vn.nom, Fsfl, kafl))

    hs1 = [x + f15 for x in dv1]
    hs2 = [x + f5 for x in dv2]
    hs3 = [x + f1 for x in dv3]
    hs4 = [x + f1 for x in dv4]
    hmin = max(max(hs1), max(hs2), max(hs3), max(hs4))
    table = pd.DataFrame({'Tipo': obs, 'Altura obstáculo': hobs, 'dVtn': dv1, 'dVts': dv2, 'dVln': dv3, 'dVls': dv4, 'htn': hs1, 'hts': hs2, 'hln': hs3, 'hls': hs4})
    display(tabulate(table, tablefmt='html', headers=['Tipo', '$h_{obs}$ (m)', '$dV_{tn}$(m)', '$dV_{ts}$ (m)', '$dV_{ln}$ (m)', '$dV_{ln}$ (m)', '$h_{tn}$ (m)', '$h_{ts}$ (m)', '$h_{ln}$ (m)', '$h_{ln}$ (m)'])) 
    print('Flechas calculadas por risco térmico: ')
    print('15%%: %.2f m' % (f15))
    print('5%%: %.2f m' % (f5))
    print('1%%: %.2f m' % (f1))
    print('Altura mínima do suporte: %.2f m' % hmin)
    print('Critério determinante: ', obs[hs1.index(hmin)])

cabo_dropdown = Dropdown(options=geral.dbCabo, description='Cabo:', value=geral.dbCabo.CAA_RAIL)
tensoes_dropdown = Dropdown(options=types.tensao, description='Tensão:', value=types.tensao._145kV)
interactive_table1 = interactive(tabelaDist, 
                                 vn=tensoes_dropdown, 
                                 cabo=cabo_dropdown, 
                                 vao=FloatSlider(description='Vão (m)', value=350, min=200, max=800, step=25),
                                 t15=FloatText(description='t₁₅ (°C)', value=59.6),
                                 t5=FloatText(description='t₅ (°C)', value=67.1),
                                 t1=FloatText(description='t₁ (°C)', value=72.9))
interactive_table1

interactive(children=(Dropdown(description='Tensão:', index=5, options=(<tensao._15kV: (13.8, 15.0)>, <tensao.…